In [22]:
import nltk 
import numpy as np
import matplotlib.pyplot as plt
import random
from nltk.corpus import stopwords
import re

In [25]:
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

In [26]:
import string

In [27]:
pos = twitter_samples.strings('positive_tweets.json')

In [28]:
# pos[:10]

In [29]:
neg = twitter_samples.strings('negative_tweets.json')

In [30]:
len(neg)

5000

In [31]:
# neg[:10]


In [87]:
Data = pos[:4000] + neg[:4000]

In [89]:
test_pos =pos[4000:]
train_pos = pos[:4000]
test_neg = neg[4000:]
train_neg = neg[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [90]:
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [91]:
labels.shape

(10000, 1)

In [92]:
def tokenize_string(tweet):
    
    '''preprocesses the texts and stems and tokenizes the string'''
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*','',tweet2)
    tweet2 = re.sub(r'#','',tweet2)
    
    tokenizer = TweetTokenizer(preserve_case = False,strip_handles=True,reduce_len=True)
    
    tokens = tokenizer.tokenize(tweet2)
    
    stop_words = stopwords.words('english')
    
    punctuation = string.punctuation
    
    clean_tokens = []
    for i in tokens:
        if (i not in stop_words and i not in punctuation):
            clean_tokens.append(i)
            
    stemmed_tokens = []
    stemmer = PorterStemmer()
    for i in clean_tokens:
        stemmed_word = stemmer.stem(i)
        stemmed_tokens.append(stemmed_word)
    return stemmed_tokens

In [93]:
tokenize_string(train_x[0]),train_x[0]

(['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)'],
 '#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)')

In [94]:
def build_freqs(tweets,labels):
    label = labels.squeeze().tolist()
    d = {}
    for x,y in zip(tweets,label):
        for z in tokenize_string(x):
            pair = (z,y)
            if pair in d:
                d[pair] +=1
            else:
                d[pair] = 1
    return d

In [95]:
l = build_freqs(train_x,train_y)

In [96]:
list(l.items())[:10]

[(('followfriday', 1.0), 23),
 (('top', 1.0), 30),
 (('engag', 1.0), 7),
 (('member', 1.0), 14),
 (('commun', 1.0), 27),
 (('week', 1.0), 72),
 ((':)', 1.0), 2847),
 (('hey', 1.0), 60),
 (('jame', 1.0), 7),
 (('odd', 1.0), 2)]

In [97]:
def train_naive_bayes(l,train_x,train_y):
    vocab = set(x[0] for x in l.keys())
    V = len(vocab)
    pos_no=neg_no = 0 
    log_prior =0 
    loglikelyhood = {}
    for x in l.keys():
        if x[1] > 0 :
            pos_no +=l[x]
        else:
            neg_no +=l[x]
    D = len(train_y)
    
    D_pos = sum(train_y)
    
    D_neg = D- D_pos
    print(D,D_pos,D_neg)
    log_prior = np.log(D_pos) - np.log(D_neg)
    
    for x in vocab:
        positive_freq = l.get((x,1),0)
        negative_feq = l.get((x,0),0)
        
        p_w_pos = (positive_freq+1)/(V+D_pos)
        p_w_neg = (negative_feq+1) /(V+D_neg)
        loglikelyhood[x] = np.log(p_w_pos/p_w_neg)
    return log_prior,loglikelyhood

In [98]:
log_prior,loglikelyhood = train_naive_bayes(l,train_x,train_y)

8000 4000.0 4000.0


In [99]:
log_prior

0.0

In [100]:
loglikelyhood

{'24': -0.9162907318741551,
 '👆': -1.0986122886681098,
 'reign': -0.6931471805599453,
 'tio': 0.6931471805599453,
 '2500k': -0.6931471805599453,
 'standard': 1.0986122886681098,
 'pussi': 0.28768207245178085,
 'ooop': -0.6931471805599453,
 'blockchain': 0.6931471805599453,
 'maaay': 0.6931471805599453,
 'inde': 1.9459101490553132,
 'ex-tweleb': 0.6931471805599453,
 'richard': -1.0986122886681098,
 'zaz': -0.6931471805599453,
 'wkend': 1.3862943611198906,
 'jd': 0.6931471805599453,
 'june': -1.0986122886681098,
 'lure': -0.6931471805599453,
 'steve': 1.0986122886681098,
 'law': -0.5108256237659907,
 'claw': 0.6931471805599453,
 'x14': 0.6931471805599453,
 'blain': -1.0986122886681098,
 'prayer': 0.0,
 'robe': -0.6931471805599453,
 'thick': 0.6931471805599453,
 'selo': 0.6931471805599453,
 'bloi': 0.6931471805599453,
 'insomnia': 0.6931471805599453,
 'custard': 0.6931471805599453,
 'ps3': 0.0,
 'khan': 0.9162907318741551,
 'paulin': 0.6931471805599453,
 'allerg': -1.3862943611198906,
 'a

In [101]:
def naive_bayes_predict(tweet,logprior,loglikelyhood):
    
    p =0 
    p+=logprior
    
    for x in tokenize_string(tweet):
        if x in loglikelyhood:
            p+=loglikelyhood[x]
    return p

In [102]:
naive_bayes_predict('hello and thankyou for your support',log_prior,loglikelyhood)

3.2203562100386347

In [103]:
train_x[0]

'#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

In [104]:
def naive_bayes_test(test_x,test_y,logprior,loglikelyhood):
    predictions =[]
    accuracy =0 
    
    for x in test_x:
        if naive_bayes_predict(x,logprior,loglikelyhood) > 0:
            predictions.append(1)
        else:
            predictions.append(0)
    error = np.mean(np.abs(predictions-test_y))
    
    accuracy = 1- error
    return accuracy

In [105]:
"accuracy is "+ str(naive_bayes_test(test_x,test_y,log_prior,loglikelyhood)*100) + '%'

'accuracy is 99.45%'

In [106]:
test_string = input()

prediction = 'positive'if naive_bayes_predict(test_string,log_prior,loglikelyhood)>0 else 'negative'

prediction


i hate the way this guy is talking


'negative'